In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./pruebaFinal/")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
        if label == 1:
            cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)

    cv2.imshow('rostros',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

class MaskDetector:
    def __init__(self, model_path='./pruebaFinal/', cascade_path='./haarcascade_frontalface_default.xml'):
        self.model = load_model(model_path)
        self.results = {0: 'without mask', 1: 'mask'}
        self.GR_dict = {0: (0, 0, 255), 1: (0, 255, 0)}
        self.rect_size = 4
        self.cap = cv2.VideoCapture(0)
        self.haarcascade = cv2.CascadeClassifier(cascade_path)

    def detect_faces(self, frame):
        frame = cv2.flip(frame, 1, 1)
        rerect_size = cv2.resize(frame, (frame.shape[1] // self.rect_size, frame.shape[0] // self.rect_size))
        faces = self.haarcascade.detectMultiScale(rerect_size)
        return [(v * self.rect_size) for v in faces]

    def process_frame(self, frame):
        faces = self.detect_faces(frame)
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            rerect_sized = cv2.resize(face_img, (150, 150))
            normalized = rerect_sized / 255.0
            reshaped = np.reshape(normalized, (1, 150, 150, 3))
            reshaped = np.vstack([reshaped])
            result = self.model.predict(reshaped)
            label = np.argmax(result, axis=1)[0]
            if label == 1:
                cv2.rectangle(frame, (x, y), (x+w, y+h), self.GR_dict[label], 2)
        return frame

    def run(self):
        while True:
            rval, im = self.cap.read()
            processed_frame = self.process_frame(im)
            cv2.imshow('rostros', processed_frame)
            key = cv2.waitKey(10)
            if key == 27:
                break

        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detector = MaskDetector()
    detector.run()
